Test model llama3

In [1]:
%pip install -q python-dotenv~=1.0.1 lamini==3.0.5

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import logging
import os
from datetime import datetime
from pprint import pprint
import sqlite3
import pandas as pd
# from utils_llm import setup_logging, get_schema, get_schema_s, make_llama_3_prompt, get_default_finetune_args, get_dataset
import random
import copy
import json
import jsonlines
from tqdm import tqdm
from typing import AsyncIterator, Iterator, Union
from lamini.generation.base_prompt_object import PromptObject
from lamini.generation.generation_node import GenerationNode
from lamini.generation.generation_pipeline import GenerationPipeline

In [7]:
from dotenv import load_dotenv
import os
import lamini

# Load environmental variable from .env file
load_dotenv()
# Get LAMINI_API_KEY with key from .env file
LAMINI_API_KEY = os.getenv("LAMINI_API_KEY")

# Token access to https://app.lamini.ai/tune/
lamini.api_key = LAMINI_API_KEY # Active API Tokens

ad3d62932bc25b48963dd3413dd94cb3414dd650fc47f735704badeb56cdf9ca


In [8]:
# Load the llm3 model
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [9]:
# Create prompt with template of llm3 model
prompt = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

Please write a graduate card for my good friend Mạnh Tiền\
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a possible graduate card for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Huge congratulations on your graduation!

You've worked hard to achieve this milestone, and it's an incredible accomplishment. I'm so proud of you for pushing through challenges and staying committed to your goals.

As you move forward, remember that this is just the beginning of an exciting new chapter in your life. You have so much to look forward to, and I have no doubt that you'll make the most of every opportunity that comes your way.

Here's to many more adventures, successes, and memories to come! Keep shining your light and making a difference in the world.

Best wishes,
[Your Name]"

Feel free to modify it as you see fit to make it more personal and special for your friend Mạnh Tiền!


In [11]:
prompt2 = (
  "<|begin_of_text|>"  # Start of prompt
  "<|start_header_id|>system<|end_header_id|>\n\n"  #  header - system
  "You are a helpful assistant."  # system prompt
  "<|eot_id|>" # end of turn
  "<|start_header_id|>user<|end_header_id|>\n\n" # header - user
  "Please write a birthday card for my good friend Mạnh Tiền"
  "<|eot_id|>" # end of turn
  "<|start_header_id|>assistant<|end_header_id|>\n\n" # header - assistant
)
prompt = prompt2

# Return result of question by llm3 model
result = llm.generate(prompt, max_new_tokens=200)
print(result)

Here's a birthday card message for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Happy birthday to an amazing friend like you! On your special day, I want to take a moment to express my gratitude for the incredible memories we've shared and the laughter we've enjoyed together. You bring so much joy and positivity to those around you, and I feel lucky to have you in my life.

Here's to another year of adventure, good health, and making more unforgettable memories together! May your birthday be as bright and wonderful as you are.

Wishing you a day filled with love, laughter, and all your favorite things.

Best regards, [Your Name]"

Feel free to modify it as you see fit!


In [ ]:
# Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt.
def make_llama_3_prompt(user, system=""):
  system_prompt = ""
  if system != "":
    system_prompt = (
      f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
    )
  return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [15]:
system_prompt = "You are a helpful assistant." # system prompt
user_prompt = "Please write a graduate card for my good friend Mạnh Tiền"

prompt = make_llama_3_prompt(user_prompt, system_prompt)

result = llm.generate(prompt, max_new_tokens=200)
print(result)


Here's a possible graduate card for your friend Mạnh Tiền:

"Dear Mạnh Tiền,

Huge congratulations on your graduation!

You've worked hard to achieve this milestone, and it's an incredible accomplishment. I'm so proud of you for pushing through challenges and staying committed to your goals.

As you move forward, remember that this is just the beginning of an exciting new chapter in your life. You have so much to look forward to, and I have no doubt that you'll make the most of every opportunity that comes your way.

Here's to many more adventures, successes, and memories to come! Keep shining your light and making a difference in the world.

Best wishes,
[Your Name]"

Feel free to modify it as you see fit to make it more personal and special for your friend Mạnh Tiền!


In [14]:
user_prompt = "Tell me a joke about birthday cake"
prompt = make_llama_3_prompt(user_prompt)

result = llm.generate(prompt, max_new_tokens=200)
print(result)

Why was the birthday cake in a bad mood?

Because it was feeling crumby!


Create a SQL Agent

In [19]:
logger = logging.getLogger(__name__)
print(logger)

engine = sqlite3.connect("./nba_roster.db")
print(engine)
# Create a cursor object using the connection
cursor = engine.cursor()

# Sample data template
teams = ['Los Angeles Lakers', 'Golden State Warriors', 'Brooklyn Nets', 'Milwaukee Bucks', 'Dallas Mavericks']
names = ['Player ' + str(i) for i in range(1, 101)]
positions = ['PG', 'SG', 'SF', 'PF', 'C']
colleges = ['College A', 'College B', 'College C', 'College D', 'College E']

# Function to generate random salary
def generate_salary():
    return f"${random.randint(1000000, 50000000):,}"

# Generating 100 rows of data
nba_data = [
    (
        random.choice(teams),
        name,
        str(random.randint(0, 99)),  # Jersey number
        random.choice(positions),
        random.randint(19, 40),  # Age
        f'{random.randint(6, 7)}-{random.randint(0, 11)}',  # Height (e.g., 6-9)
        str(random.randint(190, 260)),  # Weight
        random.choice(colleges),
        generate_salary()
    )
    for name in names
]

# Insert the generated data into the nba_roster table
cursor.executemany('''
INSERT INTO nba_roster (team, name, jersey, pos, age, ht, wt, college, salary)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', nba_data)

# Commit the changes
engine.commit()
logger.info(f"Connected to the database at './film_database.db': {engine}")

# setup_logging()

<Logger __main__ (WARNING)>


DatabaseError: file is not a database

In [17]:
def get_schema():
  return """\
  0|team|TEXT eg. "Toronto Raptors"
  1|name|TEXT eg. "Otto Porter Jr."
  2|jersey|TEXT eg. "0" and when null has a value "NA"
  3|pos|TEXT eg. "PF"
  4|age|INT eg. "22" in years
  5|ht|TEXT eg. `6' 7"` or `6' 10"`
  6|wt|TEXT eg. "232 lbs" 
  7|college|TEXT eg. "Michigan" and when null has a value "--"
  8|salary|TEXT eg. "$9,945,830" and when null has a value "--"
  """
user = """Who is the highest paid NBA player?"""

system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

prompt = make_llama_3_prompt(user, system)

print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQL query:

```sql
SELECT name, salary
FROM nba_roster
WHERE salary!= '--'
ORDER BY CAST(SUBSTR(salary, 2) AS INTEGER) DESC
LIMIT 1;
```

This query works as follows:

- `WHERE salary!= '--'` filters out the rows where the salary is unknown (i.e., '--').
- `ORDER BY CAST(SUBSTR(salary, 2) AS INTEGER) DESC` sorts the remaining rows by the salary in descending order. The `SUBSTR(salary, 2)` function extracts the numeric part of the salary string (i.e., the part after the dollar sign), and the `CAST` function converts it to an integer. The `DESC` keyword specifies that we want the highest-paid players first.
- `LIMIT 1` returns only the top row, which corresponds to the highest-paid player.

Note that this query assumes that the salary is


In [18]:
# We'd like to be able to get just SQL output so we don't have to parse the query from the model response. For this we can use structured output.
result = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(result)

# Query data from model results
df = pd.read_sql(result['sqlite_query'], con=engine)
df

{'sqlite_query': "SELECT name, salary FROM nba_roster WHERE salary!= '--' ORDER BY CAST(SUBSTR(salary, 2) AS INTEGER) DESC LIMIT 1"}


DatabaseError: Execution failed on sql 'SELECT name, salary FROM nba_roster WHERE salary!= '--' ORDER BY CAST(SUBSTR(salary, 2) AS INTEGER) DESC LIMIT 1': no such table: nba_roster

In [13]:
# Query data using Sql statements
query="""SELECT salary, name
FROM nba_roster
WHERE salary != '--'
ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC
LIMIT 1;"""
df = pd.read_sql(query, con=engine)
df

,salary,name
0,"$49,649,218",Player 50


Create an Evaluation

In [27]:
# Get data to use as seed
with open('data/gold-test-set.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

print(data)

[{'question': 'What is the 99th percentile salary in the NBA?', 'answer': '46741590', 'sql': "SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*99/100-1;"}, {'question': 'What is the 75th percentile salary in the NBA?', 'answer': '13932008', 'sql': "SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*75/100-1;"}, {'question': 'What is the 25th percentile salary in the NBA?', 'answer': '2413304', 'sql': "SELECT (CAST(REPLACE(REPLACE(SALARY, '$', ''), ',','') AS INTEGER)) as percentile FROM nba_roster WHERE SALARY!= '--' order by percentile limit 1 offset (select count(*) from nba_roster where SALARY != '--')*25/100-1;"}, {'question': 'What is the median weight in the NBA?', 'ans

In [52]:
def get_schema_ex():
    return """\
0|tean|TEXT
1|name|TEXT
2|jersey|TEXT
3|pos|TEXT
4|age|INT
5|ht|TEXT
6|wt|TEXT
7|college|TEXT
8|sagary|TEXT eg.
"""

In [54]:
question = "What is the median weight in the NBA?"

system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema_ex()}
Write a sqlite query to answer the following question. Follow instructions exactly"""

prompt = make_llama_3_prompt(question, system)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|tean|TEXT
1|name|TEXT
2|jersey|TEXT
3|pos|TEXT
4|age|INT
5|ht|TEXT
6|wt|TEXT
7|college|TEXT
8|sagary|TEXT eg.

Write a sqlite query to answer the following question. Follow instructions exactly<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the median weight in the NBA?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [56]:
generated_query = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(generated_query)

df = pd.read_sql(generated_query["sqlite_query"], con=engine)
df

{'sqlite_query': 'SELECT AVG(wt) FROM nba_roster'}


,AVG(wt)
0,225.6


In [58]:
# Try Agent Reflection to see if that can improve the query
reflection = f"Question: {question}. Query: {generated_query['sqlite_query']}. This query is invalid (gets the error Execution failed on sql 'SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL': near \"FROM\": syntax error), so it cannot answer the question. Write a corrected sqlite query."
reflection_prompt = make_llama_3_prompt(reflection, system)
print(reflection_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|tean|TEXT
1|name|TEXT
2|jersey|TEXT
3|pos|TEXT
4|age|INT
5|ht|TEXT
6|wt|TEXT
7|college|TEXT
8|sagary|TEXT eg.

Write a sqlite query to answer the following question. Follow instructions exactly<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: What is the median weight in the NBA?. Query: SELECT AVG(wt) FROM nba_roster. This query is invalid (gets the error Execution failed on sql 'SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL': near "FROM": syntax error), so it cannot answer the question. Write a corrected sqlite query.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [59]:
reflection_query = llm.generate(reflection_prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(reflection_query)

try:
  df = pd.read_sql(reflection_query["sqlite_query"], con=engine)
  df
except Exception as e:
  print(e)

{'sqlite_query': "SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL"}
Execution failed on sql 'SELECT AVG(CAST(SUBSTR(WT, INSTR(WT,'') + 1) AS INTEGER) FROM nba_roster WHERE WT IS NOT NULL': near "FROM": syntax error


In [60]:
# Look at the right answer
correct_sql = "select CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER) as percentile from nba_roster order by percentile limit 1 offset (select count(*) from nba_roster)/2;"
df_corrected = pd.read_sql(correct_sql, con=engine)
df_corrected

,percentile
0,0


In [62]:
# Evaluate over a larger dataset
path = "data/gold-test-set.jsonl"

with jsonlines.open(path) as reader:
  data = [obj for obj in reader]

In [63]:
datapoint = data[4]
datapoint

{'question': 'What is the average weight in the NBA?',
 'answer': '214.98',
 'sql': "SELECT AVG(CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER)) FROM nba_roster;"}

In [64]:
datapoint = data[7]
datapoint

{'question': 'Can you tell me how many players are in the NBA?',
 'answer': '600',
 'sql': 'select count(*) from nba_roster;'}

In [73]:
system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
system += "Consider the nba_roster table with the following schema:\n"
system += get_schema_ex() + "\n"
system += (
    "Write a sqlite SQL query that would help you answer the following question:\n"
)

user = datapoint["question"]
prompt = make_llama_3_prompt(user, system)
generated_sql = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)
print(generated_sql)
df = pd.read_sql(generated_sql['sqlite_query'], con=engine)
df

{'sqlite_query': 'SELECT COUNT(*) FROM nba_roster'}


,COUNT(*)
0,100


In [74]:
ref_sql = datapoint["sql"]
ref_df = pd.read_sql(ref_sql, con=engine)
ref_df

,count(*)
0,100


In [75]:
# Compare strings
str(df).lower() == str(ref_df).lower()

True

In [79]:
# Compare by using an LLM model
system = "Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster dataset"
system += "Respond with valid JSON {'explanation' : str, 'similar' : bool}"

user = (
    f"========== Dataframe 1 =========\n{str(df).lower()}\n\n"
)
user += (
    f"========== Dataframe 2 =========\n{str(ref_df).lower()}\n\n"
)
user += f"Can you tell me if these dataframes are similar?"

llm_similarity_prompt = make_llama_3_prompt(user, system)
print(llm_similarity_prompt)
llm_similarity = llm.generate(llm_similarity_prompt, output_type={"explanation": "str", "similar": "bool"}, max_new_tokens=200)
llm_similarity

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster datasetRespond with valid JSON {'explanation' : str, 'similar' : bool}<|eot_id|><|start_header_id|>user<|end_header_id|>

========== Dataframe 1 =========
   count(*)
0       100

========== Dataframe 2 =========
   count(*)
0       100

Can you tell me if these dataframes are similar?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




{'explanation': 'Both dataframes have the same count of 100, indicating they have the same number of rows. This suggests that they are similar',
 'similar': True}

In [6]:
class Args:
  def __init__(
    self,
    max_examples=100,
    sql_model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    gold_file_name="gold-test-set.jsonl",
    training_file_name="archive/generated_queries.jsonl",
    num_to_generate=10
  ):
    self.sql_model_name = sql_model_name
    self.max_examples = max_examples
    self.gold_file_name = gold_file_name
    self.training_file_name = training_file_name
    self.num_to_generate = num_to_generate

In [5]:
def load_gold_dataset(args):
  path = f"data/{args.gold_file_name}"
  
  with jsonlines.open(path) as reader:
    for i, obj in enumerate(reversed(list(reader))):
      if i >= args.max_examples:
        break
      yield PromptObject(prompt="", data=obj)

In [77]:
# Here's how to wrap that all up in a runnable class
class QueryStage(GenerationNode):
  def __init__(self, model_name):
    super().__init__(
      model_name=model_name,
      max_new_tokens=200,
    )

  def generate(
    self,
    prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
    *args,
    **kwargs,
  ):
    results = super().generate(
      prompt,
      output_type={"sqlite_query": "str"},
      *args,
      **kwargs,
    )
    return results

  def postprocess(self, obj: PromptObject):
    # Run both the generated and reference (Gold Dataset) SQL queries
    # Assessing whether the SQL queries succeeded in hitting the database (not correctness yet!)
    query_succeeded = False

    try:
      logger.error(f"Running SQL query '{obj.response['sqlite_query']}'")
      obj.data["generated_query"] = obj.response["sqlite_query"]
      df = pd.read_sql(obj.response["sqlite_query"], con=engine)
      obj.data['df'] = df
      logger.error(f"Got data: {df}")
      query_succeeded = True

    except Exception as e:
      logger.error(f"Failed to run SQL query: {obj.response['sqlite_query']}")

    logger.info(f"Running reference SQL query '{obj.data['sql']}'")
    df = pd.read_sql(obj.data["sql"], con=engine)
    logger.info(f"Got data: {df}")
    obj.data['reference_df'] = df

    logger.info(f"For question: {obj.data['question']}")
    logger.info(f"For query: {obj.response['sqlite_query']}")

    obj.data["query_succeeded"] = query_succeeded

  def preprocess(self, obj: PromptObject):
    new_prompt = make_llama_3_prompt(**self.make_prompt(obj.data))
    obj.prompt = new_prompt

  def make_prompt(self, data: dict):
    system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
    system += "Consider the nba_roster table with the following schema:\n"
    system += get_schema() + "\n"
    system += ("Write a sqlite SQL query that would help you answer the following question:\n")
    user = data["question"]
    return {
      "user": user,
      "system": system,
    }

In [78]:
# How to wrap compare in a class
class ScoreStage(GenerationNode):
  def __init__(self):
    super().__init__(
        model_name="meta-llama/Meta-Llama-3-8B-Instruct",
        max_new_tokens=150,
    )

  def generate(
    self,
    prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
    *args,
    **kwargs,
  ):
    logger.debug("ScoreStage Generate")
    results = super().generate(
      prompt,
      output_type={"explanation": "str", "similar": ["true", "false"]},
      *args,
      **kwargs,
    )        
    logger.debug(f"ScoreStage Results {results}")

    return results

  def preprocess(self, obj: PromptObject):
    obj.prompt = make_llama_3_prompt(**self.make_prompt(obj))
    logger.info(f"Scoring Stage Prompt:\n{obj.prompt}")

  def postprocess(self, obj: PromptObject):
    logger.info(f"Postprocess")
    obj.data['is_matching'] = self.is_matching(obj.data, obj.response)
    obj.data['explanation'] = obj.response["explanation"]
    obj.data['similar'] = obj.response["similar"] == "true"


  def is_matching(self, data, response):
    return (str(data.get('df',"None")).lower() == str(data['reference_df']).lower() or response['similar'] == "true")

  def make_prompt(self, obj: PromptObject):
    # Your evaluation model compares SQL output from the generated and reference SQL queries, using another LLM in the pipeline
    system_prompt = "Compare the following two dataframes. They are similar if they are almost identical, or if they convey the same information about the nba_roster dataset"
    system_prompt += "Respond with valid JSON {'explanation' : str, 'similar' : bool}"
    user_prompt = (
      f"========== Dataframe 1 =========\n{str(obj.data.get('df','None')).lower()}\n\n"
    )
    user_prompt += (
      f"========== Dataframe 2 =========\n{str(obj.data['reference_df']).lower()}\n\n"
    )
    user_prompt += f"Can you tell me if these dataframes are similar?"
    return {
      "system": system_prompt,
      "user": user_prompt
    }

In [79]:
class EvaluationPipeline(GenerationPipeline):
  def __init__(self, args):
    super().__init__()
    self.query_stage = QueryStage(args.sql_model_name)
    self.score_stage = ScoreStage()

  def forward(self, x):
    x = self.query_stage(x)
    x = self.score_stage(x)
    return x

In [80]:
async def run_eval(dataset, args):
  results = await run_evaluation_pipeline(dataset, args)
  print("Total results:", len(results))
  return results

async def run_evaluation_pipeline(dataset, args):
  results = EvaluationPipeline(args).call(dataset)
  result_list = []

  pbar = tqdm(desc="Saving results", unit=" results")
  async for result in results:
    result_list.append(result)
    pbar.update()
  return result_list

In [81]:
def save_eval_results(results, args):
  base_path = "./data/results"
  now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
  experiment_name = f"nba_sql_pipeline_{now}"
  experiment_dir = os.path.join(base_path, experiment_name)
  os.makedirs(os.path.join(base_path, experiment_name))

  # Write args to file
  args_file_name = f"{experiment_dir}/args.txt"
  with open(args_file_name, "w") as writer:
    pprint(args.__dict__, writer)

  def is_correct(r):
    if ((r.data["query_succeeded"] and r.data['is_matching']) or r.data["generated_query"] == r.data['sql']):
      return True
    return False

  # Write sql results and errors to file
  results_file_name = f"{experiment_dir}/sql_results.jsonl"
  with jsonlines.open(results_file_name, "w") as writer:
    for result in results:
      if not is_correct(result):
        continue
      writer.write({
        "question": result.data['question'],
        "query": result.data["generated_query"],
        "query_succeeded": result.data["query_succeeded"],
        "reference_sql": result.data['sql'],
        "df": str(result.data.get('df', 'None')),
        "reference_df": str(result.data['reference_df']),
        "is_matching": result.data['is_matching'],
        "similar": result.data['similar'],
      })

  results_file_name = f"{experiment_dir}/sql_errors.jsonl"
  with jsonlines.open(results_file_name, "w") as writer:
    for result in results:
      if is_correct(result):
        continue
      writer.write({
        "question": result.data['question'],
        "query": result.data["generated_query"],
        "query_succeeded": result.data["query_succeeded"],
        "df": str(result.data.get('df', 'None')),
        "reference_df": str(result.data['reference_df']),
        "is_matching": result.data['is_matching'],
        "similar": result.data['similar'],
      })

  # Write statistics to file
  average_sql_succeeded = sum([result.data["query_succeeded"] for result in results]) / len(results)
  average_correct = sum([result.data["query_succeeded"] and result.data['is_matching'] for result in results]) / len(results)

  file_name = f"{experiment_dir}/summary.txt"
  with open(file_name, "w") as writer:
    print(f"Total size of eval dataset: {len(results)}", file=writer)
    print(f"Total size of eval dataset: {len(results)}")
    print(f"Percent Valid SQL Syntax: {average_sql_succeeded*100}", file=writer)
    print(f"Percent Valid SQL Syntax: {average_sql_succeeded*100}")
    print(f"Percent Correct SQL Query: {average_correct*100}", file=writer)
    print(f"Percent Correct SQL Query: {average_correct*100}")

In [14]:
logger = logging.getLogger(__name__)
engine = sqlite3.connect("./nba_roster.db")
setup_logging()

args = Args(gold_file_name="gold-test-set-v2.jsonl")
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)

Running SQL query 'SELECT salary FROM nba_roster WHERE name = 'Marcus Smart';'
Got data: Empty DataFrame
Columns: [salary]
Index: []
Running SQL query 'SELECT name, salary FROM nba_roster WHERE team = 'Dallas Mavericks' AND pos = 'C' AND salary!= '--' ORDER BY CAST(SUBSTR(salary, 2) AS INTEGER) DESC LIMIT 1'
Got data:        name       salary
0  Player 5  $40,654,001
Running SQL query 'SELECT name FROM nba_roster WHERE team = 'Cleveland Cavaliers' AND salary!= '--' ORDER BY CAST(SUBSTR(salary, 2) AS INTEGER) DESC LIMIT 1'
Got data: Empty DataFrame
Columns: [name]
Index: []
Running SQL query 'SELECT name, salary FROM nba_roster WHERE team = 'Memphis Grizzlies' AND salary!= '--' ORDER BY salary DESC LIMIT 1'
Got data: Empty DataFrame
Columns: [name, salary]
Index: []
Running SQL query 'SELECT name FROM nba_roster WHERE team = 'Brooklyn Nets' AND age = (SELECT MAX(age) FROM nba_roster WHERE team = 'Brooklyn Nets')'
Got data:         name
0  Player 73
Running SQL query 'SELECT AVG(CAST(SUB

Total results: 40
Total size of eval dataset: 40
Percent Valid SQL Syntax: 60.0
Percent Correct SQL Query: 45.0


Generate data & Finetune

In [16]:
system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
system += (
    "Consider a table called 'nba_roster' with the following schema (columns)\n"
)
system += get_schema_s()
system += "Consider the following questions, and queries used to answer them:\n"

question = """What is the median weight in the NBA?"""
sql = "select CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER) as percentile from nba_roster order by percentile limit 1 offset (select count(*) from nba_roster)/2;"

system += "Question: " + question + "\n"
system += "Query: " + sql + "\n"

print(system)

You are an NBA analyst with 15 years of experience writing complex SQL queries.
Consider a table called 'nba_roster' with the following schema (columns)
  0|team|TEXT eg. "Toronto Raptors"
  1|name|TEXT eg. "Otto Porter Jr."
  2|jersey|TEXT eg. "0" and when null has a value "NA"
  3|pos|TEXT eg. "PF"
  4|age|INT eg. "22" in years
  5|ht|TEXT eg. `6' 7"` or `6' 10"` castable to int
  6|wt|TEXT eg. "232 lbs" 
  7|college|TEXT eg. "Michigan" and when null has a value "--"
  8|salary|TEXT eg. "$9,945,830" and when null has a value "--"
  Consider the following questions, and queries used to answer them:
Question: What is the median weight in the NBA?
Query: select CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER) as percentile from nba_roster order by percentile limit 1 offset (select count(*) from nba_roster)/2;



In [18]:
user = "Write two queries that are similar but different to those above.\n"
user += "Format the queries as a JSON object, i.e.\n"
user += '{ "explanation": str, "sql_query_1" : str, "sql_query_2": str }.\n'
user += "First write an explanation of why you decided to write these new queries in about 3-5 sentences, then write valid sqlite SQL queries for each of the 2 new queries. Make sure each query is complete and ends with a ;\n"

print(user)

Write two queries that are similar but different to those above.
Format the queries as a JSON object, i.e.
{ "explanation": str, "sql_query_1" : str, "sql_query_2": str }.
First write an explanation of why you decided to write these new queries in about 3-5 sentences, then write valid sqlite SQL queries for each of the 2 new queries. Make sure each query is complete and ends with a ;



In [19]:
prompt = make_llama_3_prompt(user, system)
result = llm.generate(prompt, output_type={ "explanation": "str", "sql_query_1" : "str", "sql_query_2": "str" }, max_new_tokens=200)
print(result)

{'explanation': 'I decided to write these new queries to provide more insights into the NBA roster data. The first query calculates the average height of players in the NBA, while the second query finds the most common college attended by NBA players. These queries are similar to the original query in that they involve extracting', 'sql_query_1': "SELECT AVG(CAST(SUBSTR(HT, 1, INSTR(HT,'')-1) AS INTEGER) as average_height FROM nba_roster", 'sql_query_2': 'SELECT college, COUNT(*) as count FROM nba_roster GROUP BY college ORDER BY count DESC LIMIT 1'}


In [20]:
# Func check result of the model to query the database
def check_sql_query(query):
  try:
    pd.read_sql(query, con=engine)
  except sqlite3.Error as e:
    logger.debug(f"Error in SQL query: {e}")
    return False
  
  logger.info(f"SQL query {query} is valid")
  return True

In [26]:
# check_sql_query(result["sql_query_1"])
check_sql_query(result["sql_query_2"])

     college  count
0  College C     23


In [27]:
# Wrap it all up together in a class

class ModelStage(GenerationNode):
  def __init__(self):
    super().__init__(
      model_name="meta-llama/Meta-Llama-3-8B-Instruct",
      max_new_tokens=300,
    )

  def generate(
    self,
    prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
    *args,
    **kwargs,
  ):
    prompt = self.add_template(prompt)

    results = super().generate(
      prompt,
      output_type={
        "explanation": "str",
        "sql_query_1": "str",
        "sql_query_2": "str",
      },
      *args,
      **kwargs,
    )

    return results

  async def add_template(self, prompts):
    async for prompt in prompts:
      new_prompt = make_llama_3_prompt(**self.make_prompt(prompt.data))
      yield PromptObject(prompt=new_prompt, data=prompt.data)

  async def process_results(self, results):
    async for result in results:
      if result is None:
        continue

      if result.response is None:
        continue

      logger.info("=====================================")
      logger.info(f"Generated query 1: {result.response['sql_query_1']}")
      logger.info(f"Generated query 2: {result.response['sql_query_2']}")
      logger.info("=====================================")

      if self.check_sql_query(result.response["sql_query_1"]):
        new_result = PromptObject(prompt="", data=copy.deepcopy(result.data))
        new_result.data.generated_sql_query = result.response["sql_query_1"]
        yield new_result

      if self.check_sql_query(result.response["sql_query_2"]):
        new_result = PromptObject(prompt="", data=copy.deepcopy(result.data))
        new_result.data.generated_sql_query = result.response["sql_query_2"]
        yield new_result

  def make_prompt(self, data):
    system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
    system += ( "Consider a table called 'nba_roster' with the following schema (columns)\n" )
    system += get_schema()
    system += "Consider the following questions, and queries used to answer them:\n"
    for example in data.sample:
      system += "Question: " + example["question"] + "\n"
      system += "Query: " + example["sql"] + "\n"

    # Important: generate relevant queries to your reference data
    # Ideally, close to those that are failing so you can show the model examples of how to do it right!
    user = "Write two queries that are similar but different to those above.\n"
    user += "Format the queries as a JSON object, i.e.\n"
    user += '{ "explanation": str, "sql_query_1" : str, "sql_query_2": str }.\n'

    # Next, use Chain of Thought (CoT) and prompt-engineering to help with generating SQL queries
    user += "First write an explanation of why you decided to write these new queries in about 3-5 sentences, then write valid sqlite SQL queries for each of the 2 new queries. Make sure each query is complete and ends with a ;\n"

    return {"system": system, "user": user}

  def check_sql_query(self, query):
    try:
      pd.read_sql(query, con=engine)
    except Exception as e:
      logger.debug(f"Error in SQL query: {e}")
      return False

    logger.info(f"SQL query {query} is valid")
    return True

In [28]:
system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
system += ("Consider a table called 'nba_roster' with the following schema (columns)\n")
system += get_schema() + "\n"
system += "Queries, and questions that they are used to answer:\n"

example_question = """What is the median weight in the NBA?"""
example_sql = "select CAST(SUBSTR(WT, 1, INSTR(WT,' ')) as INTEGER) as percentile from nba_roster order by percentile limit 1 offset (select count(*) from nba_roster)/2;"

system += "Question: " + example_question + "\n"
system += "Query: " + example_sql + "\n"

In [29]:
generated_sql = result["sql_query_2"]

In [30]:
user = "Now consider the following query.\n"
user += "Query: " + generated_sql + "\n"
user += "Write a question that this query could be used to answer.\n"
user += "Format your response as a JSON object, i.e.\n"
user += '{ "explanation": str, "question": str }.\n'

user += "First write an explanation in about 3-5 sentences, then write a one sentence question.\n"


In [31]:
prompt = make_llama_3_prompt(user, system)
result = llm.generate(prompt, output_type={ "explanation": "str", "question" : "str" }, max_new_tokens=200)
print(result)

{'explanation': 'This query groups the NBA players by their college and counts the number of players from each college. It then orders the results in descending order by the count and limits the output to the top 1 result. This allows us to identify the college that has produced the most NBA players. The query returns the name of the college and the number of players it has produced', 'question': 'What college has produced the most NBA players'}


In [32]:
# Wrap it all up together in a class which generates a question
# given a query
class QuestionStage(GenerationNode):
  def __init__(self):
    super().__init__(
      model_name="meta-llama/Meta-Llama-3-8B-Instruct",
      max_new_tokens=150,
    )

  def generate(
    self,
    prompt: Union[Iterator[PromptObject], AsyncIterator[PromptObject]],
    *args,
    **kwargs,
  ):
    results = super().generate(
      prompt,
      output_type={
        "explanation": "str",
        "question": "str",
      },
      *args,
      **kwargs,
    )
    return results

  def preprocess(self, obj: PromptObject):
    new_prompt = make_llama_3_prompt(**self.make_question_prompt(obj.data))
    obj.prompt = new_prompt

  def make_question_prompt(self, data):
    system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
    system += ("Consider a table called 'nba_roster' with the following schema (columns)\n")
    system += get_schema() + "\n"
    system += "Queries, and questions that they are used to answer:\n"
    for example in data.sample:
      system += "Query: " + example["sql"] + "\n"
      system += "Question: " + example["question"] + "\n"

    user = "Now consider the following query.\n"
    user += "Query: " + data.generated_sql_query + "\n"
    user += "Write a question that this query could be used to answer.\n"
    # Using Chain of Thought (CoT) again
    # This time you can do it programmatically with function calling, so you can easily extract a question out of the JSON object
    user += "Format your response as a JSON object, i.e.\n"
    user += '{ "explanation": str, "question": str }.\n'
    user += "First write an explanation in about 3-5 sentences, then write a one sentence question.\n"

    return {"system": system, "user": user}

In [63]:
class QueryGenPipeline(GenerationPipeline):
  def __init__(self):
    super().__init__()
    self.model_stage = ModelStage()
    self.question_stage = QuestionStage()

  def forward(self, x):
    x = self.model_stage(x)
    x = self.question_stage(x)
    return x

# Run query generation
async def run_query_gen_pipeline(gold_queries):
  return QueryGenPipeline().call(gold_queries)

In [60]:
# Generate N samples, for every example in the gold dataset
all_examples = []

async def load_gold_queries(args):
  path = f"data/{args.gold_file_name}"

  with jsonlines.open(path) as reader:
    global all_examples
    all_examples = [obj for obj in reader]

  sample_count = args.num_to_generate
  sample_size = 3

  random.seed(42)

  for i in range(sample_count):
    example_sample = ExampleSample(random.sample(all_examples, sample_size), i)
    yield PromptObject(prompt="", data=example_sample)

class ExampleSample:
  def __init__(self, sample, index):
    self.sample = sample
    self.index = index

In [ ]:
async def save_generation_results(results, args):
  path = f"data/training_data/{args.training_file_name}"

  pbar = tqdm(desc="Saving results", unit=" results")
  with jsonlines.open(path, "w") as writer:
    async for result in results:
      writer.write({
        "question": result.response["question"],
        "sql": result.data.generated_sql_query,
      })
      pbar.update()

    for example in all_examples:
      writer.write(example)
      pbar.update()

In [65]:
args = Args()
gold_queries = load_gold_queries(args)
results = await run_query_gen_pipeline(gold_queries)
await save_generation_results(results, args)

<async_generator object GenerationPipeline.call at 0x0000018096FBB2E0>





Saving results: 25 results [10:22, 24.91s/ results]


In [66]:
# Round of finetuning

# make_question will take the questions and queries from the training_file and embed them in the prompt below to form the training data.
def make_question(obj):
    system = "You are an NBA analyst with 15 years of experience writing complex SQL queries.\n"
    system += "Consider the nba_roster table with the following schema:\n"
    system += get_schema() + "\n"
    system += ("Write a sqlite SQL query that would help you answer the following question:\n")
    user = obj["question"]
    return {"system": system, "user": user}

In [88]:
# Get dataset data/gold-test-set.jsonl
dataset = get_dataset(args, make_question)
# Config argements
finetune_args = get_default_finetune_args()

In [89]:
# Fine-tuning model
llm.train(
  data_or_dataset_id=dataset,
  finetune_args=finetune_args,
  is_public=True, # For sharing
)

APIError: API error {'code': 501, 'error_id': '201008001164807608386151451202584318789', 'msg': 'Tuning is only supported for base models.'}

In [90]:
llm = lamini.Lamini(model_name="6820e678b0aecda564af83155542f568d76dff623c18d32390df6a609c00b5c6")

In [86]:
question = """Who is the highest paid NBA player?"""
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""
prompt = make_llama_3_prompt(question, system)
print("Question:\n", question)

print("Answer:")
print(llm.generate(prompt, max_new_tokens=200))

Question:
 Who is the highest paid NBA player?
Answer:
SELECT name FROM nba_roster WHERE salary = '$40,000,000'


In [75]:
query="SELECT salary, name FROM nba_roster WHERE salary != '--' ORDER BY CAST(REPLACE(REPLACE(salary, '$', ''), ',','') AS INTEGER) DESC LIMIT 1;"
df = pd.read_sql(query, con=engine)
df

,salary,name
0,"$49,649,218",Player 50


In [82]:
# Run the evaluation and you can see there is more valid SQL and correct queries
args = Args(sql_model_name="6820e678b0aecda564af83155542f568d76dff623c18d32390df6a609c00b5c6")
dataset = load_gold_dataset(args)
results = await run_eval(dataset, args)
save_eval_results(results, args)


Running SQL query 'SELECT AVG(CAST(REPLACE(SUBSTR(T2.salary, 3), ',', '') AS REAL)) FROM nba_roster AS T1 INNER JOIN nba_roster AS T2 ON T1.name = T2.name WHERE T1.pos = 'PF''
Got data:    AVG(CAST(REPLACE(SUBSTR(T2.salary, 3), ',', '') AS REAL))
0                                       3.521170e+06        
Running SQL query 'SELECT team, AVG(age) FROM nba_roster GROUP BY team ORDER BY AVG(age) DESC LIMIT 5'
Got data:                     team   AVG(age)
0  Golden State Warriors  31.937500
1        Milwaukee Bucks  30.833333
2     Los Angeles Lakers  30.181818
3       Dallas Mavericks  29.750000
4          Brooklyn Nets  29.750000
Running SQL query 'SELECT AVG(age) FROM nba_roster WHERE team = 'Miami Heat';'
Got data:   AVG(age)
0     None
Running SQL query 'SELECT AVG(age) FROM nba_roster'
Got data:    AVG(age)
0     30.39
Running SQL query 'SELECT AVG(age) FROM nba_roster WHERE team = 'Portland Trail Blazers';'
Got data:   AVG(age)
0     None
Running SQL query 'SELECT salary FROM nba_

Total results: 20
Total size of eval dataset: 20
Percent Valid SQL Syntax: 65.0
Percent Correct SQL Query: 30.0
